In [2]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, get_secret
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import sqlalchemy
import pytz
import psycopg2

In [6]:
campaign = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
campaign

/home/ec2-user/service_account_key.json


,campaign_id,objective,supplier_id,offer_type,active_base,churnedr_base,churnedo_base,comp_base,total_budget,sending_budget,offers_budget,consumed_budget,start_at,end_at,campaign_name
0,1,increase_nmv,171,sku_discounts,500,600,200,1000,70000,10000,60000.0,1000.0,2025-06-11 13:00:00,2025-07-01 10:00:00,None
1,2,increase_base,723,brand_specific_coupon,520,590,210,1010,65000,9000,56000.0,2000.0,2025-06-12 09:00:00,2025-07-02 10:00:00,None
2,3,increase_nmv,929,offers,510,620,205,980,72000,12000,60000.0,1500.0,2025-06-13 14:30:00,2025-07-03 11:30:00,None
3,4,increase_base,928,offers,530,610,215,1020,68000,9500,58500.0,2500.0,2025-06-14 12:00:00,2025-07-04 09:45:00,None
4,5,increase_nmv,750,quantity_discount,540,605,220,990,75000,11000,64000.0,1800.0,2025-06-15 10:00:00,2025-07-05 10:15:00,None
5,6,increase_nmv,1948,offers,560,615,225,995,73000,10500,62500.0,2300.0,2025-06-16 08:00:00,2025-07-06 11:00:00,None
6,7,increase_base,728,brand_specific_coupon,570,625,230,1015,67000,9500,57500.0,1900.0,2025-06-17 15:00:00,2025-07-07 14:00:00,None
7,8,increase_nmv,651,sku_discounts,580,635,240,1030,74000,10000,64000.0,2600.0,2025-06-18 16:00:00,2025-07-08 12:00:00,None
8,9,increase_base,267,offers,590,645,250,1040,71000,9800,61200.0,2100.0,2025-06-19 11:00:00,2025-07-09 09:00:00,None
9,10,increase_nmv,267,quantity_discount,600,655,260,1060,76000,11500,64500.0,3000.0,2025-06-20 13:00:00,2025-07-10 13:00:00,None


In [8]:
import pandas as pd

# Make sure end_at is a datetime column
campaign['end_at'] = pd.to_datetime(campaign['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
campaign_ongoing = campaign[campaign['end_at'] > now]
campaign_ongoing


,campaign_id,objective,supplier_id,offer_type,active_base,churnedr_base,churnedo_base,comp_base,total_budget,sending_budget,offers_budget,consumed_budget,start_at,end_at,campaign_name
24,24,Increase_NMV,36,offers,205,204,204,204,10000,4490,5497.0,NaN,2025-10-07 14:52:14,2025-10-14 14:52:14,pepsi man
25,25,Increase_NMV,36,offers,205,204,204,204,10000,4490,5497.0,NaN,2025-10-07 14:56:20,2025-10-14 14:56:20,pepsi man
26,26,Increase_NMV,36,offers,205,204,204,204,10000,4490,5497.0,NaN,2025-10-07 14:58:12,2025-10-14 14:58:12,pepsi man 2
27,27,Increase_NMV,36,offers,205,204,204,204,10000,4490,5497.0,NaN,2025-10-07 15:24:25,2025-10-14 15:24:25,pepsi man (pls work)
28,28,Increase_NMV,314,SKU_discounts,1,0,0,0,50,2,23.0,NaN,2025-10-08 17:50:31,2025-10-15 17:50:31,testing
29,29,Increase_NMV,314,SKU_discounts,1,0,0,0,26,2,23.0,NaN,2025-10-08 17:55:25,2025-10-15 17:55:25,testing 2


In [10]:
campaign_ids = campaign_ongoing['campaign_id'].tolist()
print(campaign_ids)


[24, 25, 26, 27, 28, 29]


In [9]:
offers_id = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
offers_id

/home/ec2-user/service_account_key.json


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
0,1,11,NaN,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Pending,2025-09-15 16:12:05,2025-09-16 02:12:05
1,2,11,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:05:20,2025-09-16 03:05:20
2,3,12,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:20:07,2025-09-16 03:20:07
3,4,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
4,5,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
5,6,14,20.0,"[189262, 600256, 42031, 40069, 652631, 21221, ...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
6,7,15,20.0,"[42031, 89747, 189262, 180832, 21221, 304614, ...",SKU_discounts,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
7,8,16,376.0,"[42031, 258853, 456325, 21221, 205682, 787231,...",SKU_discounts,Pending,2025-09-17 14:10:28,2025-09-18 00:10:28
8,9,17,739.0,"[42031, 69636, 45131, 215258, 463672, 673304, ...",SKU_discounts,Pending,2025-09-18 12:00:00,2025-09-18 22:00:00
9,11,19,110.0,"[42031, 457975, 313057, 21221, 156922, 36959, ...",SKU_discounts,Pending,2025-09-18 13:43:19,2025-09-18 23:43:19


In [11]:
offers_filtered = offers_id[offers_id['campaign_id'].isin(campaign_ids)]
offers_filtered

,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
15,16,24,36.0,"[367079, 276981, 485376, 643413, 38325, 58484,...",offers,Pending,NaT,NaT
16,17,25,36.0,"[367079, 276981, 485376, 643413, 38325, 58484,...",offers,Pending,NaT,NaT
17,18,26,36.0,"[367079, 276981, 485376, 643413, 38325, 58484,...",offers,Pending,NaT,NaT
18,19,27,36.0,"[367079, 276981, 485376, 643413, 38325, 58484,...",offers,Pending,2025-10-07 17:24:24,2025-10-10 17:24:24
19,20,28,314.0,[42031],SKU_discounts,Pending,2025-10-07 17:24:24,2025-10-10 17:24:24
20,21,29,314.0,[42031],SKU_discounts,Pending,2025-10-09 12:00:00,2025-10-09 22:00:00


In [13]:
offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
offers_ended = offers_filtered[offers_filtered['end_at'] < now]
offers_ended

/tmp/ipykernel_23502/215201034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
18,19,27,36.0,"[367079, 276981, 485376, 643413, 38325, 58484,...",offers,Pending,2025-10-07 17:24:24,2025-10-10 17:24:24
19,20,28,314.0,[42031],SKU_discounts,Pending,2025-10-07 17:24:24,2025-10-10 17:24:24
20,21,29,314.0,[42031],SKU_discounts,Pending,2025-10-09 12:00:00,2025-10-09 22:00:00


In [15]:
# Extract the values from offers_ended DataFrame first
if not offers_ended.empty:
    supplier_id_value = offers_ended['supplier_id'].iloc[0]
    retailer_id_value = offers_ended['retailer_list'].iloc[0]
    print(f"Supplier ID: {supplier_id_value}")
    print(f"Retailer ID: {retailer_id_value}")
else:
    print("offers_ended is empty!")


Supplier ID: 36.0
Retailer ID: [367079, 276981, 485376, 643413, 38325, 58484, 175597, 745158, 499253, 149714, 382449, 645487, 186815, 547344, 869671, 27621, 237895, 40069, 617635, 392385, 244352, 258853, 558319, 509156, 941106, 464565, 264667, 89747, 663541, 181462, 305493, 916358, 482768, 663062, 895235, 23149, 719051, 751907, 766394, 743448, 478823, 234795, 569925, 638356, 875310, 293805, 883815, 927974, 392436, 110176, 60021, 5351, 763930, 937466, 568670, 590450, 320690, 245441, 216436, 60307, 918419, 418803, 135562, 694949, 242212, 475973, 369761, 308255, 190279, 37348, 284169, 207857, 188182, 504455, 10783, 592472, 24414, 338758, 566448, 191229, 932820, 270595, 290651, 754738, 88271, 130541, 233781, 239083, 158984, 477645, 254469, 517685, 321624, 410682, 573376, 152056, 657737, 16877, 377625, 36959, 304614, 736055, 467351, 22731, 85106, 637634, 182457, 63010, 350485, 635627, 184595, 207119, 348545, 438747, 284876, 314221, 345522, 666180, 391093, 723591, 509202, 201955, 434893, 697

In [ ]:
snowflake_query("egypt", "")